# Det_demo

In [1]:
import glob
import math
from xml.etree import ElementTree as ET
from statistics import mean

# Function to rotate a point anticlockwise around an origin
def rotate_point(x, y, angle_degrees, origin):
    angle_radians = math.radians(-angle_degrees)
    ox, oy = origin
    qx = ox + math.cos(angle_radians) * (x - ox) + math.sin(angle_radians) * (y - oy)
    qy = oy - math.sin(angle_radians) * (x - ox) + math.cos(angle_radians) * (y - oy)
    return qx, qy

def parse_labelme_anns(base_path):
    xml_files_list = glob.glob(base_path + "/*.xml")
    # Initialize an empty dictionary to store all parsed details from multiple files
    all_files_details = {}

    # Loop through each XML data string in the list
    for xml_path in xml_files_list:
        with open(xml_path, 'r') as f:
            xml_data = f.read()
        
        # Parse XML data
        root = ET.fromstring(xml_data)
        
        # Get the filename
        filename = root.find('filename').text if root.find('filename') is not None else 'Unknown'
            
        # Initialize an empty list to store object details for this file
        file_object_details = []
        
        # Loop through each object in the XML
        for obj in root.findall('object'):
            # Check if the object has a name and a polygon
            if obj.find('name') is not None and obj.find('polygon') is not None:
                name = obj.find('name').text  # Get the class label/name
                polygon = obj.find('polygon')  # Get the polygon element
                coordinates = []  # Initialize an empty list to store coordinates
                
                # Get the rotation angle from attributes (default to 0 if not found)
                rotation_angle = float(obj.find('attributes').text.split('=')[-1]) if obj.find('attributes') is not None else 0.0
                
                # Loop through each point in the polygon
                for pt in polygon.findall('pt'):
                    x = float(pt.find('x').text)  # Get the x-coordinate and convert to float
                    y = float(pt.find('y').text)  # Get the y-coordinate and convert to float
                    coordinates.append((x, y))  # Append the (x, y) tuple to the coordinates list

                # Calculate the center of the polygon
                center_x = mean(x for x, y in coordinates)
                center_y = mean(y for x, y in coordinates)
                    
                # Rotate the coordinates around the center
                rotated_coordinates = [rotate_point(x, y, rotation_angle, (center_x, center_y)) for x, y in coordinates]

                file_object_details.append({
                    'name': name,
                    'coordinates': rotated_coordinates,
                    'rotation': rotation_angle
                })
        
        # Add this file's object details to the all_files_details dictionary
        all_files_details[filename] = file_object_details
    return all_files_details
all_files_details = parse_labelme_anns("/datasets/labelme_detdemo/default")

In [7]:
anns_path = "/datasets/detdemo/annfiles"
imgs_path = "/datasets/detdemo/images"
trainval_path = "/datasets/detdemo/trainval"
test_path = "/datasets/detdemo/test"

In [8]:
import os
trainval_imgs_path = os.path.join(trainval_path, "images")
trainval_anns_path = os.path.join(trainval_path, "annfiles")
test_imgs_path = os.path.join(test_path, "images")
test_anns_path = os.path.join(test_path, "annfiles")

os.makedirs(trainval_imgs_path, exist_ok=True)
os.makedirs(trainval_anns_path, exist_ok=True)
os.makedirs(test_imgs_path, exist_ok=True)
os.makedirs(test_anns_path, exist_ok=True)
os.makedirs(anns_path, exist_ok=True)
for filename, objects in all_files_details.items():
    if not objects:
        continue
    
    ann_file = os.path.join(anns_path, filename.replace('.jpg', '.txt'))
    lines = []
    
    for object in objects:
        line = []
        for coor in object['coordinates']:
            for item in coor:
                line.append(str(item))
        line.append(object['name'])
        line += "0\n"
        lines.append(" ".join(line))

    with open(ann_file, "w") as f:
        f.writelines(lines)

In [12]:
import glob 
import shutil
txtfiles = glob.glob(anns_path + "/*.txt")
imgfiles = [txtfile.replace("annfiles", "images").replace(".txt", ".jpg") for txtfile in txtfiles]

n = len(txtfiles)
n_train = int(n*0.9)

train_txtfiles = txtfiles[:n_train]
train_imgfiles = imgfiles[:n_train]

for train_txtfile, train_imgfile in zip(train_txtfiles, train_imgfiles):
    txtfilename = train_txtfile.split("/")[-1]
    imgfilename = train_imgfile.split("/")[-1]
    shutil.copy(train_txtfile, os.path.join(trainval_anns_path, txtfilename))
    shutil.copy(train_imgfile, os.path.join(trainval_imgs_path, imgfilename))
    
test_txtfiles = txtfiles[n_train:]
test_imgfiles = imgfiles[n_train:]

for test_txtfile, test_imgfile in zip(test_txtfiles, test_imgfiles):
    txtfilename = test_txtfile.split("/")[-1]
    imgfilename = test_imgfile.split("/")[-1]
    shutil.copy(test_txtfile, os.path.join(test_anns_path, txtfilename))
    shutil.copy(test_imgfile, os.path.join(test_imgs_path, imgfilename))


In [9]:
from imagesize import get
import glob

In [10]:
jmc = "/datasets/JMC"
images = glob.glob(jmc + "/*.png")
images_by_size = {}
sizes = set()
for image in images:
    images_by_size.setdefault(get(image), []).append(image)

In [3]:
for _, images in images_by_size.items():
    print(_, len(images))

(3072, 800) 2522
(2520, 800) 230


In [11]:
import os
new_dir  = "/datasets/JMC_merged"
os.makedirs(new_dir, exist_ok=True)

In [12]:
eight_images = list(filter(lambda x: "-8" in x, images))
one_images = list(filter(lambda x: "-1" in x, images))


In [15]:
import cv2
import numpy as np 
from PIL import Image
from tqdm import tqdm

for i in tqdm(range(0, len(eight_images), 3)):
    try:
        a = eight_images[i]
        b = eight_images[i+1]
        c = eight_images[i+2]
    except:
        break
    # Open the images
    image1 = cv2.imread(a)
    image2 = cv2.imread(b)
    image3 = cv2.imread(c)

    # Make sure they have the same width
    if image1.shape[1] != image2.shape[1] != image3.shape[1]:
        print("Images do not have the same width")
    else:
        # Get dimensions
        height1, width = image1.shape[:2]
        height2, _ = image2.shape[:2]
        height3, _ = image3.shape[:2]

        # Create a new array with the combined height
        new_image = np.zeros((height1 + height2 + height3, width, 3), dtype=np.uint8)

        # Copy the original images into this new array
        new_image[0:height1, 0:width] = image1
        new_image[height1:height1 + height2, 0:width] = image2
        new_image[height1 + height2:height1 + height2 + height3, 0:width] = image3

        new_image_name = a.split("/")[-1][:-4] + "_" + b.split("/")[-1][:-4] + "_" + c.split("/")[-1][:-4] + ".png"
        new_image_path = os.path.join(new_dir, new_image_name)
        # Save the new image
        cv2.imwrite(new_image_path, new_image)


  0%|          | 0/841 [00:00<?, ?it/s]

100%|█████████▉| 840/841 [17:20<00:01,  1.24s/it]


In [16]:
import cv2
import numpy as np 
from PIL import Image
from tqdm import tqdm

for i in tqdm(range(0, len(one_images), 3)):
    try:
        a = one_images[i]
        b = one_images[i+1]
        c = one_images[i+2]
    
    except:
        break
    # Open the images
    image1 = cv2.imread(a)
    image2 = cv2.imread(b)
    image3 = cv2.imread(c)

    # Make sure they have the same width
    if image1.shape[1] != image2.shape[1] != image3.shape[1]:
        print("Images do not have the same width")
    else:
        # Get dimensions
        height1, width = image1.shape[:2]
        height2, _ = image2.shape[:2]
        height3, _ = image3.shape[:2]

        # Create a new array with the combined height
        new_image = np.zeros((height1 + height2 + height3, width, 3), dtype=np.uint8)

        # Copy the original images into this new array
        new_image[0:height1, 0:width] = image1
        new_image[height1:height1 + height2, 0:width] = image2
        new_image[height1 + height2:height1 + height2 + height3, 0:width] = image3

        new_image_name = a.split("/")[-1][:-4] + "_" + b.split("/")[-1][:-4] + "_" + c.split("/")[-1][:-4] + ".png"
        new_image_path = os.path.join(new_dir, new_image_name)
        # Save the new image
        cv2.imwrite(new_image_path, new_image)


 99%|█████████▊| 76/77 [02:01<00:01,  1.60s/it]


In [28]:
import torch
a=torch.load("/odet_copy/datasets/detdemo.pth")


In [30]:
orig_path ="/datasets/detdemo/"
new_path = "/app/input/dataset/detdemo/"
for split in ("train", "test"):
    t = a[split]
    for i in t:
        i["image_path"] = i["image_path"].replace(orig_path, new_path)
        i["ann_path"] = i["ann_path"].replace(orig_path, new_path)
        

In [31]:
torch.save(a, "/odet_copy/datasets/detdemo.pth")

In [19]:
# pip install opencv-python tqdm numpy 
import argparse 

from tqdm import tqdm
import cv2
import numpy as np

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--path", type=str, required=True)
    args = parser.parse_args()

    means = np.zeros(3)
    stds = np.zeros(3)
    paths = os.listdir(args.path)
    for img_path in tqdm(paths, "Calculating stats"):
        img = cv2.imread(os.path.join(args.path, img_path))
        means += np.mean(img, axis=(0, 1)) # across height and width
        stds += np.std(img, axis=(0, 1))

    means /= len(paths)
    stds /= len(paths)

    # BGR to RGB
    means = means[::-1]
    stds = stds[::-1]
    
    print("RGB stats")
    print("Means:", means)
    print("Stds:", stds)

100%|██████████| 916/916 [09:19<00:00,  1.64it/s]


In [20]:
means

array([66.03297039, 66.81605603, 64.15269246])

In [21]:
stds

array([47.82983312, 45.1165196 , 44.93892323])